# Qwen3-VL-4B-Instruct Setup

Installing and testing Qwen3-VL-4B-Instruct for Dots and Boxes game.

In [ ]:
from transformers import AutoModelForImageTextToText, AutoProcessor


# Load the model
model = AutoModelForImageTextToText.from_pretrained("Qwen/Qwen3-VL-4B-Instruct", dtype="auto", device_map="auto")

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-4B-Instruct")

print("Model loaded successfully!")

In [ ]:
# Test: Ask the model about a game screenshot
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", 
             "image": "data/raw/game_1/turn_001.png"},
            {"type": "text", 
             "text": "This is a Dots and Boxes game. Describe what you see and suggest the best move. You are the red lines."},
        ],
    }
]

# Prepare and generate
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = processor.process_vision_info(messages)
inputs = processor(text=[text], images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt")
inputs = inputs.to("cuda" if model.device.type == "cuda" else "cpu")

generated_ids = model.generate(**inputs, max_new_tokens=128)
output = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(output[0])